# Preprocessing for the questions

In [1]:
import time as time
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tag.util import untag
import contractions
# import pycontractions # Alternative better package for removing contractions
from autocorrect import Speller



In [2]:
dtypes_questions = {'Id':'int32', 'Score': 'int16', 'Title': 'str', 'Body': 'str'}

In [3]:
df_questions = pd.read_csv('../Maj2NLP/data/Questions.csv', usecols=['Id', 'Score', 'Title', 'Body'], dtype=dtypes_questions, nrows=100)

In [4]:
df_questions[['Title', 'Body']] = df_questions[['Title', 'Body']]\
    .applymap(lambda x: str(x)\
              .encode("utf-8", errors='surrogatepass')\
              .decode("ISO-8859-1", errors='surrogatepass'))

In [5]:
# remove all the questions with a negative score
df_questions = df_questions[df_questions['Score'] > 0]

In [6]:
spell = Speller()
token = ToktokTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
charac = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'
stop_words = set(stopwords.words("english"))
adjective_tag_list = set(['JJ','JJR', 'JJS', 'RBR', 'RBS']) # List of Adjective's tag from nltk package

In [7]:
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      100 non-null    int32 
 1   Score   100 non-null    int16 
 2   Title   100 non-null    object
 3   Body    100 non-null    object
dtypes: int16(1), int32(1), object(2)
memory usage: 2.9+ KB


### Now let's remove HTML tags from the questions

In [8]:
# Parse question and title then return only the text
df_questions['Body'] = df_questions['Body'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
df_questions['Title'] = df_questions['Title'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())

c:\Users\bouba\anaconda3\envs\chatbot\lib\site-packages\bs4\__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


In [9]:
df_questions['Body'][13]

'In many places, (1,2,3) and [1,2,3] can be used interchangeably.\nWhen should I use one or the other, and why?\n'

### every other things to removes such as \n

In [10]:
def clean_text(text):
    text = re.sub(r"\'", "'", text) # apostrophe characters to whitespace
    text = re.sub(r"\n", " ", text) # newlines to whitespace
    text = re.sub(r"\xa0", " ", text) # non-breakable to whitespace
    text = re.sub('\s+', ' ', text) # more than one whitespace character to a single whitespace
    text = text.strip(' ')
    return text

In [11]:
df_questions['Title'] = df_questions['Title'].apply(lambda x: clean_text(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: clean_text(x))

In [12]:
df_questions['Body'][43]

'I\'m creating an ZIP file with ZipFile in Python 2.5, it works ok so far: import zipfile, os locfile = "test.txt" loczip = os.path.splitext (locfile)[0] + ".zip" zip = zipfile.ZipFile (loczip, "w") zip.write (locfile) zip.close() but I couldn\'t find how to encrypt the files in the ZIP file. I could use system and call PKZIP -s, but I suppose there must be a more "Pythonic" way. I\'m looking for an open source solution.'

In [13]:
# expand_contractions (i.e "wasn't", don't', isn't, 'i've')
def expand_contractions(text):
    text = contractions.fix(text)
    return text

In [14]:
df_questions['Title'] = df_questions['Title'].apply(lambda x: expand_contractions(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: expand_contractions(x))

In [15]:
df_questions['Body'][42]

'What is the best way to sanitise user input for a Python-based web application? Is there a single function to remove HTML characters and any other necessary characters combinations to prevent an XSS or SQL injection attack?'

## Spelling Correction

In [16]:
def autocorrect(text):
    words = token.tokenize(text)
    words_correct = [spell(w) for w in words]
    return ' '.join(map(str, words_correct)) # Return the text untokenize

In [17]:
df_questions['Title'] = df_questions['Title'].apply(lambda x: autocorrect(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: autocorrect(x))

In [18]:
df_questions['Body'][0]

"I am using the Photoshop ' s javascript API to find the fonts in a given SD. Given a font name returned by the API , I want to find the actual physical font file that that font name corresponds to on the disc. This is all happening in a python program running on SX so I guess I am looking for one of : Some Photoshop javascript A Python function An SX API that I can call from python"

In [19]:
df_questions.to_csv('../Maj2NLP/data/questions_preprocessed.csv', encoding='utf-8', errors='surrogatepass')

## Lowering

In [20]:
df_questions['Title'] = df_questions['Title'].str.lower()
df_questions['Body'] = df_questions['Body'].str.lower()

In [21]:
df_questions['Body'][0]

"i am using the photoshop ' s javascript api to find the fonts in a given sd. given a font name returned by the api , i want to find the actual physical font file that that font name corresponds to on the disc. this is all happening in a python program running on sx so i guess i am looking for one of : some photoshop javascript a python function an sx api that i can call from python"

### remove all non alphabetical characters

In [22]:
def remove_punctuation_and_number(text):
    """remove all punctuation and number"""
    return text.translate(str.maketrans(" ", " ", charac))



def remove_non_alphabetical_character(text):
    """remove all non-alphabetical character"""
    text = re.sub("[^a-z]+", " ", text) # remove all non-alphabetical character
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    return text

In [23]:
df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_non_alphabetical_character(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_non_alphabetical_character(x))

In [24]:
df_questions['Body'][0]

'i am using the photoshop s javascript api to find the fonts in a given sd given a font name returned by the api i want to find the actual physical font file that that font name corresponds to on the disc this is all happening in a python program running on sx so i guess i am looking for one of some photoshop javascript a python function an sx api that i can call from python'

In [25]:
df_questions.to_csv('../Maj2NLP/data/questions_preprocessed.csv', encoding='utf-8', errors='surrogatepass')


# Answers

In [26]:
df_answers = pd.read_csv('../Maj2NLP/data/Answers.csv', usecols=['Id', 'Score', 'Body', 'ParentId'],
                           dtype=dtypes_questions, nrows=100)
df_answers[['Body']] = df_answers[['Body']] \
    .applymap(lambda x: str(x) \
              .encode("utf-8", errors='surrogatepass') \
              .decode("ISO-8859-1", errors='surrogatepass'))
# remove all the questions with a negative score
df_answers = df_answers[df_answers['Score'] > 0]
spell = Speller()
token = ToktokTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
charac = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'
stop_words = set(stopwords.words("english"))
adjective_tag_list = set(['JJ', 'JJR', 'JJS', 'RBR', 'RBS'])  # List of Adjective's tag from nltk package
df_answers.info()

df_answers['Body'] = df_answers['Body'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())

df_answers['Body'] = df_answers['Body'].apply(lambda x: clean_text(x))

df_answers['Body'] = df_answers['Body'].apply(lambda x: expand_contractions(x))

df_answers['Body'] = df_answers['Body'].apply(lambda x: autocorrect(x))
## Lowering
df_answers['Body'] = df_answers['Body'].str.lower()

df_answers['Body'] = df_answers['Body'].apply(lambda x: remove_non_alphabetical_character(x))
df_answers.to_csv('../Maj2NLP/data/answers_preprocessed.csv', encoding='utf-8', errors='surrogatepass')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 0 to 99
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        83 non-null     int32 
 1   ParentId  83 non-null     int64 
 2   Score     83 non-null     int16 
 3   Body      83 non-null     object
dtypes: int16(1), int32(1), int64(1), object(1)
memory usage: 2.4+ KB
